In [1]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import AutoTokenizer, AutoModel
from utils.MIND import MIND, MIND_news, MIND_history
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [ ]:
# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")
# t3 = AutoTokenizer.from_pretrained('google/bigbird-roberta-base', cache_dir=config.path + "bert_cache/")

In [2]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'
# config.reducer = 'first'

# config.signal_length = 10
config.scale = 'large'
config.impr_size = 1000
config.mode = 'encode'
config.news = None
# config.fast = True
# config.news = 21563

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
# file_directory = manager.path + "MIND/MIND{}_{}/".format(manager.scale, manager.mode)
file_directory = manager.path + "MIND/MIND{}_dev/".format(manager.scale)
# manager.gather_same_user_impr()

a = MIND_history(manager, file_directory)
# b = MIND_news(manager)

[2021-11-05 05:29:09,615] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDlarge_dev/1000/behaviors.pkl
[2021-11-05 05:29:12,983] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDlarge_dev/news.pkl
[2021-11-05 05:29:13,922] INFO (utils.utils) deduplicating...


In [4]:
a[1]

{'impr_index': 2,
 'user_id': 516861,
 'his_id': array([62326, 56055, 50011, 56063, 14115, 18785, 39544, 19156, 27416,
        52335,  8319, 14915,   940,   872, 18691, 23304, 45940, 49597,
        56185, 13217, 21563, 58333, 35183,  9354, 30137, 44644, 47782,
        55564, 12334, 34692, 53057, 49667, 38773,  5266, 27909, 53101,
        46648, 27154, 41911, 37117,  3043, 45472, 47277, 29900, 56863,
        46850, 56989, 34069, 30705, 45279]),
 'his_encoded_index': array([[  101,  3340,  2040, ...,     0,     0,     0],
        [  101,  2450,  1010, ...,     0,     0,     0],
        [  101,  2152,  2082, ...,     0,     0,     0],
        ...,
        [  101,  1018,  2152, ...,     0,     0,     0],
        [  101,  2125,  1011, ...,  1996, 20587,  2056],
        [  101,  1015,  2757, ...,     0,     0,     0]]),
 'his_attn_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 

In [20]:
f = open('/data/v-pezhang/Data/MIND/MINDlarge_dev/behaviors.tsv','r')
count = 1
for line in f:
    if count == 127187:
        x = line.split('\t')
        print(line)
    count += 1
f.close()

162371	U213018	11/15/2019 3:31:11 AM	N104299 N113326 N99768 N126397 N47349 N15819 N30521 N97343 N123633 N36663 N67779 N85005 N454 N10711 N80686 N112324 N121098 N98238 N29941 N116801 N36246 N32721 N25818 N42209 N47193 N94659 N90686 N25894 N6195 N47118 N12252 N78429 N100233 N21707 N6799 N88769 N65757 N41300 N80126 N79834 N122561 N25463 N91580 N120705 N4289 N115671 N94872 N2927 N106025 N20441 N78936 N58184 N101718 N66190 N125867 N26043 N67061 N46137 N9892 N67133 N128175 N97753 N47193 N82603 N96616 N103634 N93049 N17355 N44524 N92646 N61712 N114848 N113688 N38388 N5674 N44148 N11266 N38845 N114568 N34519 N10923 N38845 N9690 N43402 N99458 N24327 N93987 N57217 N18966 N71977 N119590 N54618 N50729 N93493 N84799 N44826 N99305 N113698 N23386 N578 N33512 N44176 N41377 N24446 N48926 N71650 N126608 N75220 N91968 N88466 N9651 N115658 N92277 N90939 N49747 N1713 N85594 N58220 N63476 N102145 N119891 N114787 N26948 N82649 N102489 N107322 N95066 N84423 N37530 N31323 N84436 N98042	N97743-0 N114057-0 N7564

In [21]:
len(x[-1].split())

1187

In [26]:
uindexes[127186]

120778

In [29]:
imprs[127188]

(127187, [65537, 67421, 65556, 66259, 69492], [1, 0, 0, 0, 0])

In [ ]:
# manager.construct_whole_dataset()

In [2]:
behaviors = pickle.load(open('data/cache/bert/MINDlarge_dev/1000/behaviors.pkl','rb'))
# news = pickle.load(open('data/cache/bert/MINDdemo_train/news.pkl','rb'))
# news = pickle.load(open('data/cache/bert/MINDlarge_train/news.pkl','rb'))
# enc = news['encoded_news']

In [3]:
uindexes = behaviors['uindexes']
histories = behaviors['histories']
imprs = behaviors['imprs']

In [31]:
uindexes[224411]

195921

In [ ]:
len("N77613 N5324 N88129 N26834 N121794 N85484 N16078 N85722 N74147 N99932 N67397 N121551 N82799 N67770 N9375 N65119 N9375 N124922 N70796 N127337 N85056 N127659 N126802 N62383 N88290 N115743 N122766 N60882 N29907 N8427 N25353 N45946 N122920 N26947 N107290 N77265 N108319".split())

In [ ]:
news = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/bert/MINDdemo_train/news.pkl', 'rb'))
ids = news['encoded_news']
subwords = news['subwords_all']

index = 37405
news = t.convert_ids_to_tokens(a.encoded_news_original[index])
mask = a.attn_mask_original[index]
subword = a.subwords_original[index]

for i,j,k in zip(news,mask,subword):
    print(i,j,k)

In [ ]:
# bm25, entity, first
index = 1

# news = t.convert_ids_to_tokens(a[index]['his_encoded_index'][1])
# mask = a[index]['his_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
# subword = a[index]['his_subword_index'][1]

# for i,j,k in zip(news,mask,subword):
#     print(i,j,k)
res = a[index]
ids = res['cdd_id'][1]
news = t.convert_ids_to_tokens(res['cdd_encoded_index'][1])
mask = res['cdd_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
# subword = res['cdd_subword_index'][1]
print(ids)
for i,j,k in zip(news,mask):
    print(i,j,k)

In [ ]:
news = t.convert_ids_to_tokens(a[0]['his_encoded_index'][1])
mask = a[0]['his_attn_mask'][1]
word = manager.convert_tokens_to_words(news)
# subword = a[0]['his_subword_index'][1]
dedup = a[0]['his_refined_mask'][1]

for i,j,k in zip(news,mask,dedup):
    print(i,j,k)